# PCA Script Mode

In [31]:
prefix = 'Scikit-pca'

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [32]:
import pandas as pd
preprocessed_train_path_file = 's3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/transformtrain-train-output/sagemaker-scikit-learn-2020-08-12-01-21-2020-08-12-01-24-10-589/train.csv.out'
local_preprocessed_train_path_file = 'train.csv.out'
churn_df = pd.read_csv(local_preprocessed_train_path_file, header=None)
churn_df.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.0,0.119414,-0.596238,1.744368,0.978957,-0.028993,-0.893185,-0.801703,-1.982529,-1.530559,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,-1.852652,-0.596238,0.140284,-0.310405,0.970689,-0.689888,0.146389,1.232901,0.124852,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,1.181295,-0.596238,1.835130,0.185503,0.030988,-0.639063,1.568529,-0.063643,-0.846802,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.776769,-0.596238,0.216227,0.334276,0.136954,1.393914,1.394712,-0.634123,0.844596,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,-0.234547,1.508734,-0.459859,0.483049,-0.230929,0.224952,1.056954,0.921730,-0.810815,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [33]:
# churn_df.info()

In [34]:
train_y = churn_df.iloc[:,0]
train_X = churn_df.iloc[:,1:]

In [35]:
print("Shape of train_X: ", train_X.shape)
print("Shape of train_y: ", train_y.shape)

Shape of train_X:  (2333, 69)
Shape of train_y:  (2333,)


In [36]:
from sklearn import datasets
import os
import numpy as np

# iris = datasets.load_iris()
# train_X = iris.data
# train_y = iris.target

os.makedirs('./data', exist_ok =True)
np.savetxt('./data/iris.csv', train_X, delimiter=',',
           # fmt='%1.3f, %1.3f, %1.3f, %1.3f'
           fmt='%1.3f'
          )


In [37]:
WORK_DIRECTORY = 'data'
train_input = sagemaker_session.upload_data(WORK_DIRECTORY,
                                            key_prefix="{}/{}".format(prefix, WORK_DIRECTORY)
                                           )

In [38]:
print("train_input: ", train_input)

train_input:  s3://sagemaker-us-east-2-057716757052/Scikit-pca/data


In [39]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = 'pca_script_train.py'

instance_type = 'local'

sklearn = SKLearn(
    entry_point = script_path,
    framework_version = FRAMEWORK_VERSION,
    train_instance_type = instance_type,
    role = role,
#     sagemaker_session = sagemaker_session, # Exclude in local mode
    hyperparameters = {'n_components' : 2}
)

In [40]:
sklearn.fit({'train' : train_input}, wait=True)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Creating tmpsnuxl0tf_algo-1-c5qfz_1 ... 
Attaching to tmpsnuxl0tf_algo-1-c5qfz_12mdone
algo-1-c5qfz_1  | 2020-08-12 04:16:43,825 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
algo-1-c5qfz_1  | 2020-08-12 04:16:43,827 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-c5qfz_1  | 2020-08-12 04:16:43,835 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-c5qfz_1  | 2020-08-12 04:16:43,954 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-c5qfz_1  | 2020-08-12 04:16:43,963 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-c5qfz_1  | 2020-08-12 04:16:43,972 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-c5qfz_1  | 2020-08-12 04:16:43,982 sagemaker-training-toolkit INFO     Invoking user script
algo-1-c5qfz_1  | 
algo-1-c5qfz_1  | Training Env:
algo-1-c5q

In [41]:
print("model data: ", sklearn.model_data)

model data:  s3://sagemaker-us-east-2-057716757052/sagemaker-scikit-learn-2020-08-12-04-16-42-001/model.tar.gz


In [42]:
instance_type = 'local'


script_predictor = sklearn.deploy(
    initial_instance_count = 1,
    instance_type = instance_type,
)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


Attaching to tmpl120j6l7_algo-1-4ma7x_1
algo-1-4ma7x_1  | 2020-08-12 04:17:15,396 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-4ma7x_1  | 2020-08-12 04:17:15,398 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-4ma7x_1  | 2020-08-12 04:17:15,399 INFO - sagemaker-containers - nginx config: 
algo-1-4ma7x_1  | worker_processes auto;
algo-1-4ma7x_1  | daemon off;
algo-1-4ma7x_1  | pid /tmp/nginx.pid;
algo-1-4ma7x_1  | error_log  /dev/stderr;
algo-1-4ma7x_1  | 
algo-1-4ma7x_1  | worker_rlimit_nofile 4096;
algo-1-4ma7x_1  | 
algo-1-4ma7x_1  | events {
algo-1-4ma7x_1  |   worker_connections 2048;
algo-1-4ma7x_1  | }
algo-1-4ma7x_1  | 
algo-1-4ma7x_1  | http {
algo-1-4ma7x_1  |   include /etc/nginx/mime.types;
algo-1-4ma7x_1  |   default_type application/octet-stream;
algo-1-4ma7x_1  |   access_log /dev/stdout combined;
algo-1-4ma7x_1  | 
algo-1-4ma7x_1  |   upstream gunicorn {
algo-1-4ma7x_1  |     server unix:/tmp/guni

In [56]:
train_X.iloc[0,:].values.reshape(1,-1)

array([[ 0.1194137 , -0.59623803,  1.74436806,  0.97895705, -0.02899291,
        -0.8931854 , -0.8017032 , -1.98252864, -1.53055893, -0.60087965,
        -0.44388848,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0. 

In [59]:
sample = train_X.iloc[0,:].values.reshape(1,-1)
print("Shape of sample: ", sample.shape)
sample
# sample = train_X.iloc[0,:].reshape(1,-1) # Single Sample (1,-1)

Shape of sample:  (1, 69)


array([[ 0.1194137 , -0.59623803,  1.74436806,  0.97895705, -0.02899291,
        -0.8931854 , -0.8017032 , -1.98252864, -1.53055893, -0.60087965,
        -0.44388848,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0. 

In [60]:
pca_components = script_predictor.predict(sample)

algo-1-4ma7x_1  | 2020-08-12 04:25:15,607 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-4ma7x_1  | 2020-08-12 04:25:15,974 INFO - root - predict_fn: input_data - '[[ 0.1194137  -0.59623803  1.74436806  0.97895705 -0.02899291 -0.8931854
algo-1-4ma7x_1  |   -0.8017032  -1.98252864 -1.53055893 -0.60087965 -0.44388848  0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    1.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.       

In [61]:
print("pca_components: ", pca_components)

pca_components:  [[-0.82314245 -0.10872395]]


In [64]:
sample = train_X.iloc[0:2,:].values.reshape(-1,69)
print("Shape of sample: ", sample.shape)
sample
# sample = train_X.iloc[0,:].reshape(1,-1) # Single Sample (1,-1)

Shape of sample:  (2, 69)


array([[ 0.1194137 , -0.59623803,  1.74436806,  0.97895705, -0.02899291,
        -0.8931854 , -0.8017032 , -1.98252864, -1.53055893, -0.60087965,
        -0.44388848,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0. 

In [65]:
pca_components = script_predictor.predict(sample)

algo-1-4ma7x_1  | 2020-08-12 04:28:58,834 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
algo-1-4ma7x_1  | 2020-08-12 04:28:59,201 INFO - root - predict_fn: input_data - '[[ 0.1194137  -0.59623803  1.74436806  0.97895705 -0.02899291 -0.8931854
algo-1-4ma7x_1  |   -0.8017032  -1.98252864 -1.53055893 -0.60087965 -0.44388848  0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    1.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.          0.          0.          0.
algo-1-4ma7x_1  |    0.          0.          0.       

In [66]:
pca_components

array([[-0.82314245, -0.10872395],
       [-0.34342986,  0.09079016]])